In [15]:
import numpy as np
import cv2

def zero_order_hold_enlargement(image, scale_factor):
    height, width, channels = image.shape

    new_height = int(height * scale_factor)
    new_width = int(width * scale_factor)

    output = np.zeros((new_height, new_width, channels), dtype=np.uint8)
    kernel = np.ones((int(scale_factor), int(scale_factor)))

    for c in range(channels):
        for i in range(new_height):
            for j in range(new_width):
                x = int(i / scale_factor)
                y = int(j / scale_factor)
                output[i, j, c] = image[x, y, c]

    return output

input_image = cv2.imread('img.jpg')
scale_factor = 2

output_image = zero_order_hold_enlargement(input_image, scale_factor)

output_image_path = 'zero_order_hold_image.jpg'
cv2.imwrite(output_image_path, output_image)
cv2.imshow('Original Image', input_image)
cv2.waitKey(0)
cv2.imshow('Zero-Order Hold Enlargement with Convolution', output_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [13]:
import numpy as np
import cv2

def first_order_hold_enlargement(image, scale_factor):
    height, width, channels = image.shape

    new_height = int(height * scale_factor)
    new_width = int(width * scale_factor)

    output = np.zeros((new_height, new_width, channels), dtype=np.uint8)

    mask = np.array([[0.25, 0.5, 0.25],
                      [0.5, 1.0, 0.5],
                      [0.25, 0.5, 0.25]])

    for c in range(channels):
        for i in range(new_height):
            for j in range(new_width):
                x = i / scale_factor
                y = j / scale_factor
                x0 = int(x)
                y0 = int(y)
                x1 = min(x0 + 1, height - 1)
                y1 = min(y0 + 1, width - 1)
                dx = x - x0
                dy = y - y0
                interpolated_value = (1 - dx) * (1 - dy) * image[x0, y0, c] + \
                                     dx * (1 - dy) * image[x1, y0, c] + \
                                     (1 - dx) * dy * image[x0, y1, c] + \
                                     dx * dy * image[x1, y1, c]
                
                output[i, j, c] = int(interpolated_value)

    return output

input_image = cv2.imread('img.jpg')

scale_factor = 2


output_image = first_order_hold_enlargement(input_image, scale_factor)

output_image_path = 'first_order_hold_image.jpg'
cv2.imwrite(output_image_path, output_image)
cv2.imshow('Original Image', input_image)
cv2.waitKey(0)
cv2.imshow('First-Order Hold Enlargement with Convolution', output_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np

def arbitrary_order_interpolation(image, k):
    height, width, channels = image.shape
    new_height = (height - 1) * k + 1
    new_width = (width - 1) * k + 1
    interpolated_image = np.zeros((new_height, new_width, channels), dtype=np.uint8)

    for i in range(height):
        for j in range(width - 1):
            a = image[i, j]
            b = image[i, j + 1]
            c = (b - a) / k
            min_val = np.min([a, b], axis=0)  

            for t in range(1, k):
                if i * k + t < new_height:
                    interpolated_image[i * k + t, j * k : (j + 1) * k] = min_val + t * c

        interpolated_image[i * k : (i + 1) * k, (width - 1) * k] = image[i, width - 1]

    return interpolated_image


input_image = cv2.imread("img.jpg")
k = 2  
interpolated_image = arbitrary_order_interpolation(input_image, k)


cv2.imwrite("arbitrary_interpolated_img.jpg", interpolated_image)

cv2.imshow("Original Image", input_image)
cv2.waitKey(0)
cv2.imshow("Interpolated Image", interpolated_image)
cv2.waitKey(0)
cv2.destroyAllWindows()